In [2]:
import pandas as pd
import quantaxis_ext
from QUANTAXIS import QA_fetch_stock_day_adv
from QUANTAXIS import QA_fetch_index_day_adv
import ploter
import talib
import numpy as np
import settings
import units
import header
import calculator as calc
import matplotlib.pyplot as plt

#设定绘图的默认大小
import matplotlib
matplotlib.rcParams["figure.figsize"]=[16,5]

#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Noto Sans CJK SC','SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [3]:
START_VALS = 1000000 # 初始投资总额
START = '2018-01-01' # 初始投资日期
END = '2018-12-31' # 结束投资日期
# 投资组合：贵州茅台、中国平安、招商银行、格力电器
SYMBOLS = ['600519','601318','600036','000651'] 
# 比率分配
ALLOCS = [0.4,0.4,0.1,0.1]

def _test_index(code):
    """测试指数数据是否在本地能读取到
    
    Return:
        如果数据读取正常，则返回True，否则返回False
    """
    return not QA_fetch_index_day_adv(code,START,END).data.empty

ZS_CODE=['000300','399300']#指数代码。沪深300
for z in ZS_CODE:
    if _test_index(z):
        ZS_CODE=z
        break
if not isinstance(ZS_CODE, str):
    raise AssertionError
print(ZS_CODE)

000300


## 日回报率

In [4]:
daily_return=quantaxis_ext.fetch_index_stock_daily_adv(SYMBOLS, ZS_CODE, START, END).pct_change()[1:]
daily_return.head()

,zs_000300,600519,601318,600036,000651
date,,,,,
2018-01-03,0.005869,0.017063,-0.022317,0.011816,0.011074
2018-01-04,0.004237,0.029629,0.002959,-0.010677,0.020372
2018-01-05,0.002407,0.001750,-0.004074,0.015177,0.027909
2018-01-08,0.005173,0.018649,-0.012414,-0.020930,-0.001044
2018-01-09,0.007005,0.040405,0.026568,0.010180,0.032825


## 组合的日回报率

In [5]:
symbol_daily_return=daily_return.drop(columns=['zs_'+ZS_CODE])
symbol_daily_return.head()

,600519,601318,600036,000651
date,,,,
2018-01-03,0.017063,-0.022317,0.011816,0.011074
2018-01-04,0.029629,0.002959,-0.010677,0.020372
2018-01-05,0.001750,-0.004074,0.015177,0.027909
2018-01-08,0.018649,-0.012414,-0.020930,-0.001044
2018-01-09,0.040405,0.026568,0.010180,0.032825


## 组合日收益协方差矩阵

In [6]:
symbol_cov_mat=symbol_daily_return.cov()
symbol_cov_mat.head()

,600519,601318,600036,000651
600519,0.000506,0.000297,0.000225,0.000374
601318,0.000297,0.000396,0.000284,0.000322
600036,0.000225,0.000284,0.000370,0.000249
000651,0.000374,0.000322,0.000249,0.000532


## 组合日收益均值

In [7]:
symbol_avg_rets =symbol_daily_return.mean()
symbol_avg_rets.head()

600519   -0.000476
601318   -0.000867
600036   -0.000483
000651   -0.000706
dtype: float64

## 按照默认的组合分配比率，默认的初始资金，计算期末时的价值

In [8]:
port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,ALLOCS,START_VALS)

In [9]:
print('按照最默认组合 {0}\n计算后的期末余额为 {1}。\n比期初降低 {2:.2%}。'.format(ALLOCS,
                                                        port_vals.iloc[-1].sum(),
                                                        port_vals.iloc[-1].sum()/START_VALS-1))

按照最默认组合 [0.4, 0.4, 0.1, 0.1]
计算后的期末余额为 808563.1559890351。
比期初降低 -19.14%。


## 计算最小方差组合（长期投资，比率不可为负）

In [10]:
import portfolioopt as opt

In [11]:
weight_min_portfolio=opt.min_var_portfolio(symbol_cov_mat)
weight_min_portfolio

600519    0.220303
601318    0.200134
600036    0.506971
000651    0.072591
dtype: float64

In [12]:
min_portfolio_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_min_portfolio.values,START_VALS)

In [13]:
print('按照最小方差组合（长期投资，比率不可为负） {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_min_portfolio.values,
                                                        min_portfolio_port_vals.iloc[-1].sum(),
                                                        min_portfolio_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

按照最小方差组合（长期投资，比率不可为负） [0.22030341 0.20013408 0.50697133 0.07259118]
计算后的期末余额为 828043.0164388939
比默认组合增加 2.41%。


## 计算最小方差组合（短期投资，比率可为负）

In [14]:
weight_min_portfolio_short=opt.min_var_portfolio(symbol_cov_mat,allow_short=True)
weight_min_portfolio_short

600519    0.220360
601318    0.200183
600036    0.506984
000651    0.072472
dtype: float64

In [15]:
min_portfolio_short_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_min_portfolio_short.values,START_VALS)

In [16]:
print('按照最小方差组合（短期投资，比率可为负） {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_min_portfolio_short.values,
                                                        min_portfolio_short_port_vals.iloc[-1].sum(),
                                                        min_portfolio_short_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

按照最小方差组合（短期投资，比率可为负） [0.22036029 0.20018344 0.50698394 0.07247233]
计算后的期末余额为 828045.6236738007
比默认组合增加 2.41%。


## 计算Markowitz组合

In [17]:
weight_markowitz=opt.markowitz_portfolio(symbol_cov_mat,symbol_avg_rets,symbol_avg_rets.quantile(0.7))
weight_markowitz

600519    0.339792
601318    0.000262
600036    0.654976
000651    0.004970
dtype: float64

In [18]:
markowitz_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_markowitz.values,START_VALS)

In [19]:
print('按照Markowitz组合 {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_markowitz.values,
                                                        markowitz_port_vals.iloc[-1].sum(),
                                                        markowitz_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

按照Markowitz组合 [3.39792269e-01 2.62319821e-04 6.54975655e-01 4.96975577e-03]
计算后的期末余额为 846203.7053146708
比默认组合增加 4.66%。


## 计算“切线资产组合” (tangency portfolio)

In [20]:
weight_tangency=opt.tangency_portfolio(symbol_cov_mat, symbol_avg_rets)
weight_tangency

ValueError: domain error

In [ ]:
tangency_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_tangency.values,START_VALS)

In [ ]:
print('按照切线资产组合 {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_tangency.values,
                                                        tangency_port_vals.iloc[-1].sum(),
                                                        tangency_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))